In [1]:
import sys
import os

# Add the repo root (one level up from notebooks) to Python path
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)


# Now imports work
from options_pricing.black_scholes import black_scholes_price
from options_pricing.monte_carlo import monte_carlo_option_price
from data.data_loader import fetch_stock_data, compute_annualized_volatility, get_latest_price


import yfinance as yf

# Fetch stock data
ticker = "AAPL"
data = fetch_stock_data(ticker, start="2024-01-01", end="2025-01-01")

# Spot price & volatility as scalars
S0 = float(get_latest_price(data))
sigma = float(compute_annualized_volatility(data))

# Parameters
K = 200
T = 0.5
r = 0.05

# Price options
call_price = black_scholes_price(S0, K, T, r, sigma, option_type="call")
put_price = monte_carlo_option_price(S0, K, T, r, sigma,
                                       option_type="put", n_simulations=100000, seed=42)

print(f"Spot Price: {S0:.2f}, Volatility: {sigma:.2%}")
print(f"Call Price (BS): {call_price:.2f}")
print(f"Put Price (MC): {put_price:.2f}")



[*********************100%***********************]  1 of 1 completed

Spot Price: 249.53, Volatility: 22.32%
Call Price (BS): 55.36
Put Price (MC): 0.89


In [2]:
import sys
import os

# Add the repo root (one level up from notebooks) to Python path
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
from options_pricing.finite_difference import finite_difference

S0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2

price_explicit = finite_difference(S0, K, T, r, sigma, method="explicit")
price_implicit = finite_difference(S0, K, T, r, sigma, method="implicit")
price_cn = finite_difference(S0, K, T, r, sigma, method="crank-nicolson")

print("Explicit:", price_explicit)
print("Implicit:", price_implicit)
print("Crank-Nicolson:", price_cn)


Explicit: 10.448637487496628
Implicit: 10.361641084463221
Crank-Nicolson: 10.362771893255863


In [ ]:
import yfinance as yf
import pandas as pd

def get_free_float_checked(tickers):
    """
    Fetch shares outstanding, free float, and free float % for tickers,
    and check for inconsistencies.
    
    Parameters:
        tickers (list): List of ticker symbols
    
    Returns:
        pd.DataFrame: DataFrame with columns 
                      ['Ticker', 'Shares Outstanding', 'Free Float', 'Free Float %', 'Valid Data']
    """
    results = []

    for symbol in tickers:
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            shares_outstanding = info.get("sharesOutstanding", None)
            free_float = info.get("floatShares", None)
            
            # Calculate free float %
            if shares_outstanding and free_float:
                free_float_pct = round((free_float / shares_outstanding) * 100, 2)
                valid = free_float <= shares_outstanding
            else:
                free_float_pct = None
                valid = False
            if (valid):
                results.append({
                    "Ticker": symbol,
                    "Shares Outstanding": shares_outstanding,
                    "Free Float": free_float,
                    "Free Float %": free_float_pct,
                    "Valid Data": valid
                })
        except Exception as e:
            print(f"Error fetching {symbol}: {e}")
            results.append({
                "Ticker": symbol,
                "Shares Outstanding": None,
                "Free Float": None,
                "Free Float %": None,
                "Valid Data": False
            })
    
    return pd.DataFrame(results)


# Example usage
tickers = ["GOOG", "AAPL", "MSFT", "TSLA"]
df = get_free_float_checked(tickers)
print(df)


  Ticker  Shares Outstanding   Free Float  Free Float %  Valid Data
0   GOOG          5430000128  10836344940        199.56       False
1   AAPL         14840399872  14814270914         99.82        True
2   MSFT          7433169920   7422388288         99.85        True
3   TSLA          3225449984   2711473609         84.06        True
